# Wokflow  **TEST** con Full Bayesiana 

##  Big Picture

Este workflow es de testeo, hace la predicción final en 202106 el último mes para el cual podemos construir la clase_ternaria completa y por lo tanto medir la ganancia.

Se muestra el workflow con la Bayesian Optimization diseñada para que LightGBM maximice la ganancia en pesos argentinos
<br> En la Primera Competencia se maximizo la metrica global ROC AUC,  ahora pasamos a la metrica real que llamamos ganancia y que solo se concentra en los  11000 registros con mayor probabilidad de  {"BAJA+1","BAJA+2"}


Intencionalmente este notebook es un *semi-esqueleto*  al que usted deberá completar reutilizando código que ya generó para la Primera Competencia y nuevo código que resuelva las problemáticas presentes en esta competencia.

## Inicializacion

In [45]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-16 18:34:33 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2737528,146.2,7992996,426.9,9991244,533.6
Vcells,8931001,68.2,18961217418,144662.7,23649445693,180431.0


In [46]:
PARAM <- list()
PARAM$experimento <- "apo-001-final-250"
PARAM$semilla_primigenia <- 123457

In [47]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [48]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-16 18:34:39 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2738287,146.3,7992996,426.9,9991244,533.6
Vcells,725590728,5535.9,15168973935,115730.1,23649445693,180431.0


[1] "2025-11-16 18:35:15 UTC"

In [49]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

Completar a gusto LUEGO de realizar un analisis exploratorio de datos.
<br> No necesariamente en esta Segunda Competencia conviele eliminar los mismos campos que en la Primera ...

In [50]:
# Salsa Magica para 202106
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

### Data Quality

Se deben reparar los atributos del dataset que para un cierto mes TODOS sus valores son cero.
<br> Relevar en forma muy minuciosa en el dataset cuales son los  <atributo,mes> que estan dañados.
<br> Algunas alternativas de solución son:
* No hacer absolutamente nada, dejar el valor 0 tal cual está, a sabiendas que es incorrecto
* Reemplazar esos valores dañados por  NA
* Interpolar cada valor dañado por el valor del mes previo y el posterior
* Calcularlo a partir de un modelo, libreria  MICE

a este codigo de Data Quality  lo debera escribir usted

### Data Drifting

Se debe corregir el drifting natural que ocurre en loa datos, en particular los datos monetarios que se vieron fuertemente afectados por una alta inflación
<br> Posibles métodos son:
* No hacer absolutamente nada
* Ajuste de valores monetarios por indices del tipo :
   * IPC  Indice de Precios al Consumidor
   * Dolar Oficial
   * Dolar Blue
   * UVA  Unidad de Valor Adquisitivo

a este codigo de Data Drifting lo debera escribir usted

In [51]:
# función para imputar NAs en meses con promedio 0
ReemplazarCerosPorNA <- function() {
      # obtener lista de columnas numéricas
    numeric_cols <- names(dataset)[sapply(dataset, is.numeric)]
     # recorrer columnas
  for (col in numeric_cols) {
    # calcular promedio por mes
    promedios <- dataset[, .(prom = mean(get(col), na.rm = TRUE)), by = foto_mes]
# identificar los meses con promedio = 0
    meses_cero <- promedios[prom == 0, foto_mes]

    # reemplazar por NA en esos meses
    if (length(meses_cero) > 0) {
      dataset[foto_mes %in% meses_cero, (col) := NA]
    }
  }
}


In [52]:
ReemplazarCerosPorNA()

In [53]:
numeric_cols <- names(dataset)[sapply(dataset, is.numeric)]
numeric_cols

[1] "numero_de_cliente"                   
  [2] "foto_mes"                            
  [3] "active_quarter"                      
  [4] "cliente_vip"                         
  [5] "internet"                            
  [6] "cliente_edad"                        
  [7] "cliente_antiguedad"                  
  [8] "mrentabilidad"                       
  [9] "mrentabilidad_annual"                
 [10] "mcomisiones"                         
 [11] "mactivos_margen"                     
 [12] "mpasivos_margen"                     
 [13] "cproductos"                          
 [14] "tcuentas"                            
 [15] "ccuenta_corriente"                   
 [16] "mcuenta_corriente_adicional"         
 [17] "mcuenta_corriente"                   
 [18] "ccaja_ahorro"                        
 [19] "mcaja_ahorro"                        
 [20] "mcaja_ahorro_adicional"              
 [21] "mcaja_ahorro_dolares"                
 [22] "cdescubierto_preacordado"            
 [23] "mcuentas_saldo"                      
 [24] "ctarjeta_debito"                     
 [25] "ctarjeta_debito_transacciones"       
 [26] "mautoservicio"                       
 [27] "ctarjeta_visa"                       
 [28] "ctarjeta_visa_transacciones"         
 [29] "mtarjeta_visa_consumo"               
 [30] "ctarjeta_master"                     
 [31] "ctarjeta_master_transacciones"       
 [32] "mtarjeta_master_consumo"             
 [33] "cprestamos_prendarios"               
 [34] "mprestamos_prendarios"               
 [35] "cprestamos_hipotecarios"             
 [36] "mprestamos_hipotecarios"             
 [37] "cplazo_fijo"                         
 [38] "mplazo_fijo_dolares"                 
 [39] "mplazo_fijo_pesos"                   
 [40] "cinversion1"                         
 [41] "minversion1_pesos"                   
 [42] "minversion1_dolares"                 
 [43] "cinversion2"                         
 [44] "minversion2"                         
 [45] "cseguro_vida"                        
 [46] "cseguro_auto"                        
 [47] "cseguro_vivienda"                    
 [48] "cseguro_accidentes_personales"       
 [49] "ccaja_seguridad"                     
 [50] "cpayroll_trx"                        
 [51] "mpayroll"                            
 [52] "mpayroll2"                           
 [53] "cpayroll2_trx"                       
 [54] "ccuenta_debitos_automaticos"         
 [55] "mcuenta_debitos_automaticos"         
 [56] "ctarjeta_visa_debitos_automaticos"   
 [57] "mttarjeta_visa_debitos_automaticos"  
 [58] "ctarjeta_master_debitos_automaticos" 
 [59] "mttarjeta_master_debitos_automaticos"
 [60] "cpagodeservicios"                    
 [61] "mpagodeservicios"                    
 [62] "cpagomiscuentas"                     
 [63] "mpagomiscuentas"                     
 [64] "ccajeros_propios_descuentos"         
 [65] "mcajeros_propios_descuentos"         
 [66] "ctarjeta_visa_descuentos"            
 [67] "mtarjeta_visa_descuentos"            
 [68] "ctarjeta_master_descuentos"          
 [69] "mtarjeta_master_descuentos"          
 [70] "ccomisiones_mantenimiento"           
 [71] "mcomisiones_mantenimiento"           
 [72] "ccomisiones_otras"                   
 [73] "mcomisiones_otras"                   
 [74] "cforex"                              
 [75] "cforex_buy"                          
 [76] "mforex_buy"                          
 [77] "cforex_sell"                         
 [78] "mforex_sell"                         
 [79] "ctransferencias_recibidas"           
 [80] "mtransferencias_recibidas"           
 [81] "ctransferencias_emitidas"            
 [82] "mtransferencias_emitidas"            
 [83] "cextraccion_autoservicio"            
 [84] "mextraccion_autoservicio"            
 [85] "ccheques_depositados"                
 [86] "mcheques_depositados"                
 [87] "ccheques_emitidos"                   
 [88] "mcheques_emitidos"                   
 [89] "ccheques_depositados_rechazados"   

In [54]:
drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}


In [55]:
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios

[1] "mrentabilidad"                       
 [2] "mrentabilidad_annual"                
 [3] "mcomisiones"                         
 [4] "mactivos_margen"                     
 [5] "mpasivos_margen"                     
 [6] "mcuenta_corriente_adicional"         
 [7] "mcuenta_corriente"                   
 [8] "mcaja_ahorro"                        
 [9] "mcaja_ahorro_adicional"              
[10] "mcaja_ahorro_dolares"                
[11] "mcuentas_saldo"                      
[12] "mautoservicio"                       
[13] "mtarjeta_visa_consumo"               
[14] "mtarjeta_master_consumo"             
[15] "mprestamos_prendarios"               
[16] "mprestamos_hipotecarios"             
[17] "mplazo_fijo_dolares"                 
[18] "mplazo_fijo_pesos"                   
[19] "minversion1_pesos"                   
[20] "minversion1_dolares"                 
[21] "minversion2"                         
[22] "mpayroll"                            
[23] "mpayroll2"                           
[24] "mcuenta_debitos_automaticos"         
[25] "mttarjeta_visa_debitos_automaticos"  
[26] "mttarjeta_master_debitos_automaticos"
[27] "mpagodeservicios"                    
[28] "mpagomiscuentas"                     
[29] "mcajeros_propios_descuentos"         
[30] "mtarjeta_visa_descuentos"            
[31] "mtarjeta_master_descuentos"          
[32] "mcomisiones_mantenimiento"           
[33] "mcomisiones_otras"                   
[34] "mforex_buy"                          
[35] "mforex_sell"                         
[36] "mtransferencias_recibidas"           
[37] "mtransferencias_emitidas"            
[38] "mextraccion_autoservicio"            
[39] "mcheques_depositados"                
[40] "mcheques_emitidos"                   
[41] "mcheques_depositados_rechazados"     
[42] "mcheques_emitidos_rechazados"        
[43] "matm"                                
[44] "matm_other"                          
[45] "Master_mfinanciacion_limite"         
[46] "Master_msaldototal"                  
[47] "Master_msaldopesos"                  
[48] "Master_msaldodolares"                
[49] "Master_mconsumospesos"               
[50] "Master_mconsumosdolares"             
[51] "Master_mlimitecompra"                
[52] "Master_madelantopesos"               
[53] "Master_madelantodolares"             
[54] "Master_mpagado"                      
[55] "Master_mpagospesos"                  
[56] "Master_mpagosdolares"                
[57] "Master_mconsumototal"                
[58] "Master_mpagominimo"                  
[59] "Visa_mfinanciacion_limite"           
[60] "Visa_msaldototal"                    
[61] "Visa_msaldopesos"                    
[62] "Visa_msaldodolares"                  
[63] "Visa_mconsumospesos"                 
[64] "Visa_mconsumosdolares"               
[65] "Visa_mlimitecompra"                  
[66] "Visa_madelantopesos"                 
[67] "Visa_madelantodolares"               
[68] "Visa_mpagado"                        
[69] "Visa_mpagospesos"                    
[70] "Visa_mpagosdolares"                  
[71] "Visa_mconsumototal"                  
[72] "Visa_mpagominimo"

In [56]:
drift_rank_cero_fijo(campos_monetarios)

inicio drift_rank_cero_fijo()
mrentabilidad  mrentabilidad_annual  mcomisiones  mactivos_margen  mpasivos_margen  mcuenta_corriente_adicional  mcuenta_corriente  mcaja_ahorro  mcaja_ahorro_adicional  mcaja_ahorro_dolares  mcuentas_saldo  mautoservicio  mtarjeta_visa_consumo  mtarjeta_master_consumo  mprestamos_prendarios  mprestamos_hipotecarios  mplazo_fijo_dolares  mplazo_fijo_pesos  minversion1_pesos  minversion1_dolares  minversion2  mpayroll  mpayroll2  mcuenta_debitos_automaticos  mttarjeta_visa_debitos_automaticos  mttarjeta_master_debitos_automaticos  mpagodeservicios  mpagomiscuentas  mcajeros_propios_descuentos  mtarjeta_visa_descuentos  mtarjeta_master_descuentos  mcomisiones_mantenimiento  mcomisiones_otras  mforex_buy  mforex_sell  mtransferencias_recibidas  mtransferencias_emitidas  mextraccion_autoservicio  mcheques_depositados  mcheques_emitidos  mcheques_depositados_rechazados  mcheques_emitidos_rechazados  matm  matm_other  Master_mfinanciacion_limite  Master_msaldoto

In [57]:
ncol(dataset)

[1] 153

In [58]:
names(dataset)

[1] "numero_de_cliente"                        
  [2] "foto_mes"                                 
  [3] "active_quarter"                           
  [4] "cliente_vip"                              
  [5] "internet"                                 
  [6] "cliente_edad"                             
  [7] "cliente_antiguedad"                       
  [8] "cproductos"                               
  [9] "tcuentas"                                 
 [10] "ccuenta_corriente"                        
 [11] "ccaja_ahorro"                             
 [12] "cdescubierto_preacordado"                 
 [13] "ctarjeta_debito"                          
 [14] "ctarjeta_debito_transacciones"            
 [15] "ctarjeta_visa"                            
 [16] "ctarjeta_visa_transacciones"              
 [17] "ctarjeta_master"                          
 [18] "ctarjeta_master_transacciones"            
 [19] "cprestamos_prendarios"                    
 [20] "cprestamos_hipotecarios"                  
 [21] "cplazo_fijo"                              
 [22] "cinversion1"                              
 [23] "cinversion2"                              
 [24] "cseguro_vida"                             
 [25] "cseguro_auto"                             
 [26] "cseguro_vivienda"                         
 [27] "cseguro_accidentes_personales"            
 [28] "ccaja_seguridad"                          
 [29] "cpayroll_trx"                             
 [30] "cpayroll2_trx"                            
 [31] "ccuenta_debitos_automaticos"              
 [32] "ctarjeta_visa_debitos_automaticos"        
 [33] "ctarjeta_master_debitos_automaticos"      
 [34] "cpagodeservicios"                         
 [35] "cpagomiscuentas"                          
 [36] "ccajeros_propios_descuentos"              
 [37] "ctarjeta_visa_descuentos"                 
 [38] "ctarjeta_master_descuentos"               
 [39] "ccomisiones_mantenimiento"                
 [40] "ccomisiones_otras"                        
 [41] "cforex"                                   
 [42] "cforex_buy"                               
 [43] "cforex_sell"                              
 [44] "ctransferencias_recibidas"                
 [45] "ctransferencias_emitidas"                 
 [46] "cextraccion_autoservicio"                 
 [47] "ccheques_depositados"                     
 [48] "ccheques_emitidos"                        
 [49] "ccheques_depositados_rechazados"          
 [50] "ccheques_emitidos_rechazados"             
 [51] "tcallcenter"                              
 [52] "ccallcenter_transacciones"                
 [53] "thomebanking"                             
 [54] "chomebanking_transacciones"               
 [55] "ccajas_transacciones"                     
 [56] "ccajas_consultas"                         
 [57] "ccajas_depositos"                         
 [58] "ccajas_extracciones"                      
 [59] "ccajas_otras"                             
 [60] "catm_trx"                                 
 [61] "catm_trx_other"                           
 [62] "ctrx_quarter"                             
 [63] "tmobile_app"                              
 [64] "cmobile_app_trx"                          
 [65] "Master_delinquency"                       
 [66] "Master_status"                            
 [67] "Master_Fvencimiento"                      
 [68] "Master_Finiciomora"                       
 [69] "Master_fultimo_cierre"                    
 [70] "Master_fechaalta"                         
 [71] "Master_cconsumos"                         
 [72] "Master_cadelantosefectivo"                
 [73] "Visa_delinquency"                         
 [74] "Visa_status"                              
 [75] "Visa_Fvencimiento"                        
 [76] "Visa_Finiciomora"                         
 [77] "Visa_fultimo_cierre"                      
 [78] "Visa_fechaalta"                           
 [79] "Visa_cconsumos"                           
 [80] "Visa_cadelantosefectivo"                  
 [

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [59]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll_rank / cliente_edad]

Sys.time()

[1] "2025-11-16 18:38:21 UTC"

### Feature Engineering Historico

In [60]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

In [61]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [62]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [63]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

[1] "2025-11-16 18:39:59 UTC"

In [64]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [65]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 921

[1] "2025-11-16 18:40:43 UTC"

In [66]:
vars <- c(
"ctrx_quarter_normalizado",
"mcaja_ahorro_rank",
"ctrx_quarter",
"mtarjeta_visa_consumo_rank",
"mcuenta_corriente_rank",
"Visa_status",
"cpayroll_trx",
"foto_mes",
"mcuentas_saldo_rank",
"mpayroll_rank",
"mactivos_margen_rank",
"cdescubierto_preacordado",
"Visa_Finiciomora",
"Visa_status_tend6",
"ctarjeta_visa_transacciones",
"cdescubierto_preacordado_tend6",
"mpayroll_sobre_edad",
"mpasivos_margen_rank",
"Visa_mpagominimo_rank",
"ccomisiones_mantenimiento_tend6"
)


In [67]:
for(a in vars){
 for(b in vars){
  if(a != b){
   nuevo_nombre <- paste0(a, "_sobre_", b)
   dataset[, (nuevo_nombre) := get(a) / fifelse(get(b)==0, NA_real_, get(b))]
  }
 }
}

In [68]:
for(a in vars){
 for(b in vars){
  if(a != b){
   nuevo_nombre <- paste0(a, "_por_", b)
   dataset[, (nuevo_nombre) := get(a) * get(b)]
  }
 }
}

### Feature Engineering a partir de hojas de Random Forest

### Reduccion dimensionalidad con canaritos asesinos

Intencionalmente este código no se brinda
<br> El objetivo de esto es reducir la dimensionalidad del dataset unicamente para que la Bayesian Optimization corra mas rápido
<br> Tambien se puede probar con Boruta

## Modelado

### Target Engineering

In [69]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)]

### Training Strategy

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202108  obviamente completo

* final_train =  [201901, 202106] sin undersampling de los CONTINUA

* training
   * testing = 202106
   * training = [201901, 202104]  donde se consideran el 5% de los CONTINUA

In [70]:
PARAM$trainingstrategy$testing <- c(202106)

PARAM$trainingstrategy$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102,202103,202104
)

PARAM$trainingstrategy$undersampling <- 1


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [71]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

#### Registros  cambio las proporciones de POS/NEG

In [72]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar := runif(nrow(dataset))]
dataset[, training := 0L]

dataset[  foto_mes %in%  PARAM$trainingstrategy$training &
  (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]


### Optimizacion de Hipeparámetros

Se optimizan los hiperparámetros maximizando la ganancia.

In [73]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

In [74]:
dtrain <- lgb.Dataset(
  data= data.matrix(dataset[training == 1L, campos_buenos, with = FALSE]),
  label= dataset[training == 1L, clase01],
  free_raw_data= TRUE
)

cat("filas", nrow(dtrain), "columnas", ncol(dtrain), "\n")
Sys.time()

filas 4060310 columnas 1680 


[1] "2025-11-16 18:47:01 UTC"

In [75]:
# defino los datos de testing
dataset_test <- dataset[foto_mes %in% PARAM$trainingstrategy$testing]

# precalculo el campo de la ganancia
dataset_test[, gan := -20000.0 ]
dataset_test[ clase_ternaria=="BAJA+2", gan := 780000]

# precalculo la test_matrix
test_matrix <- data.matrix(dataset_test[, campos_buenos, with= FALSE])

cat("filas", nrow(dataset_test), "columnas", ncol(dataset_test), "\n")
Sys.time()

filas 164313 columnas 1685 


[1] "2025-11-16 18:47:07 UTC"

In [76]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

In [77]:
# Especificacion de la cantidad de iteraciones de la Bayesian Optimization
# 50 es razonable
PARAM$hipeparametertuning$BO_iteraciones <- 10 # un 50 seria mas razonable

In [78]:
# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "custom",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  extra_trees = FALSE,

  max_depth = -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split = 0.0, # min_gain_to_split >= 0.0
  min_sum_hessian_in_leaf = 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1 = 0.0, # lambda_l1 >= 0.0
  lambda_l2 = 0.0, # lambda_l2 >= 0.0

  bagging_fraction = 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction = 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction = 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance = FALSE, #
  scale_pos_weight = 1.0, # scale_pos_weight > 0.0

  drop_rate = 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop = 50, # <=0 means no limit
  skip_drop = 0.5, # 0.0 <= skip_drop <= 1.0

  max_bin= 31
)

In [79]:
# Notar que se recorren algunos hiperparametros en forma logaritmica
#   y que con forbidden se tiene en cuenta el juego que hay entre min_data_in_leaf y num_leaves

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeNumericParam("num_iterations", lower= 0.0, upper= 11.1, trafo= function(x) as.integer( round(2^x)) ),
  makeNumericParam("learning_rate", lower= -8.0, upper= -1.0, trafo= function(x) 2^x ),
  makeNumericParam("feature_fraction", lower= 0.05, upper= 1.0 ),
  makeNumericParam("min_data_in_leaf", lower= 0.0, upper= log2(nrow(dtrain)/2), trafo= function(x) as.integer(round(2^x)) ),
  makeNumericParam("num_leaves", lower= 1.0, upper= 10.0, trafo= function(x) as.integer(round(2^x)) ),
  forbidden= quote( (2^min_data_in_leaf)*(2^num_leaves) > nrow(dtrain) )
)

El parámetro **ksemillerio** indica se se hace semillerio DENTRO de la bayesiana
* 1 **no** se hace Ensemble Semillerio, apenas se corre un solo LightGBM
* mayor a 1, se hace un  k-Ensemble Semillerio

El parámetro  **repe** indica si dentro de la bayesiana se toman varias medidas y luego se promedian
<br> Esto se hace ya sea que se llama a un solo LightGBM o se haceun Ensemble Semillerio de LightGBMs
<br> Tener en cuenta que repe multiplica linealmente el tiempo de corrida de la Bayesian Optimization

In [80]:
PARAM$hipeparametertuning$ksemillerio <- 1L
PARAM$hipeparametertuning$repe <- 1L


In [81]:
if(!require("primes")) install.packages("primes")
require("primes")

In [82]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$BO$semillas <- sample(primos)[seq( PARAM$hipeparametertuning$ksemillerio * PARAM$hipeparametertuning$repe )]

cat( PARAM$BO$semillas)

597889

In [83]:
if(!require("rlist")) install.packages("rlist")
require("rlist")

In [84]:
# logueo al archivo BO_log.txt
loguear  <- function( reg, arch=NA, verbose=TRUE )
{
  t0 <- Sys.time()
  archivo <- arch
  if( is.na(arch) ) archivo <- paste0( folder, substitute( reg), ext )


  if( !file.exists( archivo ) )
  {
    # Escribo los titulos
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  # escribo el registro
  linea  <- paste0( format(t0, "%Y%m%d.%H%M%S"),  "\t",     # la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  # grabo al archivo

  if( verbose )  cat( linea )   # imprimo por pantalla
}

In [85]:
# esto esta en una funcion para que el garbage collector lo libere
# entrena, aplica el modelo a testing, y devuelve el vector de probabilidades

OneTrainPredict <- function(param_completo) {
    
  modelo <- lgb.train(
    data= dtrain,
    param= param_completo
  )
  gmodelo <<- modelo

  # aplico el modelo a los datos nuevos
  pred <- predict(
    modelo,
    test_matrix
  )

  return( pred )
}

In [86]:
# En el argumento x llegan los parametros de la bayesiana
#  devuelve la ganancia en datos de testing

# aqui se ira guardando la mejor iteracion de la bayesiana
gmejor <- list()
gmejor$iter <- 0
gmejor$gan <- -Inf

giter <- 0
if( file.exists("BO_log.txt") ){
  tb_BO <- fread("BO_log.txt")
  giter <- nrow(tb_BO) -1 
}

EstimarGanancia_lightgbm <- function(x) {

  giter <<- giter + 1
  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  vgan_mesetas <- c()  # las ganancias, tengo repe de ellas

  # loop de las repeticionies
  for( repe in seq( PARAM$hipeparametertuning$repe ) )
  {
     desde <- (repe-1)*PARAM$hipeparametertuning$ksemillerio + 1
     hasta <- desde + PARAM$hipeparametertuning$ksemillerio -1
     rsemillas <- PARAM$BO$semillas[ desde:hasta ]

     # vector inicial de probabilidades
     vpred_acum <- rep( 0.0, nrow(dataset_test) )

     # loop del semillerio
     for( sem in rsemillas ) # itero semillerio
     {
        param_completo$seed <- sem  # asigno se semilla
        vpred_acum <- vpred_acum + OneTrainPredict( param_completo )
        
        gc(full= TRUE, verbose= FALSE)
     }

     # Calculo de ganancia suavizada de la meseta
     tb_prediccion <- dataset_test[, list(gan)]
     tb_prediccion[, prob := vpred_acum ]

     setorder(tb_prediccion, -prob)
     tb_prediccion[, gan_acum := cumsum(gan)]

     # la meseta es un punto, mil para la izquierda, otros mil para la derecha
     tb_prediccion[, gan_meseta :=
       frollmean(
         x= gan_acum, n= 2001, align= "center",
         na.rm= TRUE, hasNA= TRUE
      )
     ]

     vgan_mesetas <- c(vgan_mesetas, tb_prediccion[, max(gan_meseta, na.rm = TRUE)] )
  }

  gan_mesetas_prom <- mean( vgan_mesetas ) 

  if( gan_mesetas_prom > gmejor$gan ){
    gmejor$gan <<- gan_mesetas_prom
    gmejor$iter <<- giter

    # hrabo importancia de variables
    fwrite( lgb.importance(gmodelo),
      file= paste0("impo_", giter, ".txt"),
      sep= "\t"
    )
  }

  # datos qeu voy a loguear
  xx <- copy(param_completo)
  xx$iter <- giter
  xx$metrica_mejor <- gmejor$gan
  xx$metrica_sd <- sd(vgan_mesetas)
  xx$metrica <- gan_mesetas_prom

  loguear( xx, "BO_log.txt")
  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")  # le reordeno a mlrMBO

  return( gan_mesetas_prom ) #tiempo_corrida) )
}

In [87]:
# Aqui comienza la configuracion de la Bayesian Optimization
#  es compleja la configuracion de una Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hipeparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hipeparametertuning$BO_iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)

Sys.time()

[1] "2025-11-16 18:47:07 UTC"

#### Corrida de la Bayesian Optimization

In [ ]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Sys.time()

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Obviamente los datos donde se aplica el modelo es el mes  {202108} , que no tiene clase
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202106]
* Se realiza undersampling al 10%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}, , esta es una *enorme* decisión.

### Final Training Strategy

In [ ]:
PARAM$train_final$future <- c(202108)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104,202105, 202106
)

PARAM$train_final$undersampling <- 1

In [ ]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

In [ ]:
# canaritos
plocal <- list()
plocal$qcanaritos <- 5L

PARAM$train_final$lgbm$qcanaritos <- plocal$qcanaritos

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
  setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )
}

Sys.time()

#### Registros cambio las proporciones de POS/NEG

In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Adaptacion Hiperparametros Optimos

Solamente escalo min_data_in_leaf  por  nrow(dataset_train_final) / nrow(dtrain)

In [ ]:
# leo el archivo donde quedaron los hiperparametros optimos
tb_BO <-  fread("BO_log.txt")
setorder( tb_BO, -metrica)  # ordeno por metrica descendente
tb_BO[1]

In [ ]:
# en la tabla ademas de los parametros del LightGBM, hay campos de salida
param_lgbm <- union( names(PARAM$lgbm$param_fijos),  names(PARAM$hipeparametertuning$hs$pars) )

PARAM$train_final$param_mejores <- as.list( tb_BO[1, param_lgbm, with=FALSE])

PARAM$train_final$param_mejores$min_data_in_leaf <- as.integer( round(PARAM$train_final$param_mejores$min_data_in_leaf * nrow(dataset_train_final[training == 1L]) / nrow(dtrain)))

cat( tb_BO[1, min_data_in_leaf] , PARAM$train_final$param_mejores$min_data_in_leaf, "\n")
PARAM$train_final$param_mejores

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [ ]:
# Semillerio Final
PARAM$train_final$APO <- 10
PARAM$train_final$ksemillerio  <- 10

PARAM$train_final$cortes <- c(10500, 11000, 11500, 12000)

In [ ]:
# valvula de seguridad anti atajos
if( PARAM$train_final$APO * PARAM$train_final$ksemillerio  < 100
  | PARAM$train_final$APO < 5
)  {
  # stop("No quiera overfitear")
  cat("No quiera overfitear")
}

In [ ]:
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$ksemillerio*PARAM$train_final$APO )]
PARAM$train_final$semillas

In [ ]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

In [ ]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$param_mejores)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [ ]:
# dfuture: solo el mes que querés predecir, sin tocar clase_ternaria
dfuture <- dataset[ foto_mes %in% PARAM$train_final$future ]

cols0 <- copy(colnames(dfuture))
filas <- nrow(dfuture)

# canaritos si los usaste en entrenamiento
if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dfuture[, paste0("canarito_",i) := runif( filas) ]
  }
  cols_canaritos <- copy( setdiff( colnames(dfuture), cols0 ) )
  setcolorder( dfuture, c( cols_canaritos, cols0 ) )
}

cat("se agegaron canaritos")


mfuture <- data.matrix(dfuture[, campos_buenos, with = FALSE])

# acá voy a acumular las predicciones del ENSEMBLE global
vpred_ens <- rep(0.0, nrow(dfuture))
qmodelos  <- 0L

# carpeta para guardar un archivo por meta_modelo
dir.create("predicciones_meta_modelos", showWarnings = FALSE)

for( vapo in seq(PARAM$train_final$APO) ) {

  # predicción acumulada SOLO para este meta_modelo (puede tener varias seeds)
  vpred_mm <- rep(0.0, nrow(dfuture))
  qacum_mm <- 0L

  desde    <- 1 + (vapo - 1L) * PARAM$train_final$ksemillerio
  hasta    <- desde + PARAM$train_final$ksemillerio - 1L
  semillas <- PARAM$train_final$semillas[desde:hasta]

  for( sem in semillas ) {
    arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
    if( file.exists( arch_modelo ) ) {
      modelo_final <- lgb.load(arch_modelo)
      p <- predict(modelo_final, mfuture)

      vpred_mm <- vpred_mm + p
      qacum_mm <- qacum_mm + 1L

      rm(modelo_final)
      gc()
    }
  }

  # si este meta_modelo efectivamente predijo algo
  if( qacum_mm > 0L ) {
    # promedio interno de sus seeds
    vpred_mm <- vpred_mm / qacum_mm

    # sumo al ensemble global
    vpred_ens <- vpred_ens + vpred_mm
    qmodelos  <- qmodelos + 1L

    # guardo archivo SOLO de este meta_modelo
    tb_mm <- dfuture[, .(numero_de_cliente, foto_mes)]
    tb_mm[, meta_modelo := vapo ]
    tb_mm[, prob := vpred_mm ]

    archivo_mm <- sprintf("predicciones_meta_modelos/prediccion_meta_modelo_%02d.txt", vapo)
    fwrite(tb_mm, file = archivo_mm, sep = "\t")

    rm(tb_mm)
    gc()
  }
}

if( qmodelos == 0L ) stop("No se cargó ningún modelo, algo está mal con las semillas.")

# promedio final del ENSEMBLE entre meta_modelos
vpred_ens <- vpred_ens / qmodelos

dfuture[, prob_ens := vpred_ens ]

# ordeno por probabilidad descendente UNA sola vez
setorder(dfuture, -prob_ens)

# cortes que querés probar
cortes <- c(9000L, 9500L, 10000L, 10500L, 11000L, 11500L, 12000L)

dir.create("kaggle", showWarnings = FALSE)

for (envios in cortes) {
  # por las dudas, no romper si el corte es más grande que la cantidad de filas
  envios_efectivos <- min(envios, nrow(dfuture))

  # genero la columna Predicted para este corte
  dfuture[, Predicted := 0L]
  if (envios_efectivos > 0L) {
    dfuture[1:envios_efectivos, Predicted := 1L]
  }

  # nombre del archivo según el corte
  archivo_kaggle <- sprintf("./kaggle/KA_%s_%d.csv",
                            PARAM$experimento, envios)

  # grabo el archivo para este corte
  fwrite(
    dfuture[, .(numero_de_cliente, Predicted)],
    file = archivo_kaggle,
    sep  = ","
  )

  cat("Generado:", archivo_kaggle,
      "con", envios_efectivos, "envíos\n")
}

Sys.time()


In [ ]:
#archivos_modelos <- list.files("/content/buckets/b1/exp/apo-001/modelitos", full.names = TRUE)

#imp_acum <- data.table(Feature = character(), Gain_sum = double(), n = integer())

#total_modelos <- length(archivos_modelos)

#for(i in seq_along(archivos_modelos)) {
# cat("Voy por el modelo", i, "de", length(archivos_modelos), "\n")
# modelo <- lgb.load(archivos_modelos[i])
# imp <- lgb.importance(model = modelo)
# imp <- as.data.table(imp[, c("Feature", "Gain")])
# setnames(imp, "Gain", "Gain_sum")
# imp[, n := 1L]

# if(nrow(imp_acum) == 0) {
#  imp_acum <- imp
# } else {
#  imp_acum <- rbind(imp_acum, imp)[, .(
#   Gain_sum = sum(Gain_sum),
#   n = sum(n)
#  ), by = Feature]
# }

# rm(modelo, imp)
# gc()
#}

#imp_global <- imp_acum[, .(Gain_prom = Gain_sum / n), by = Feature]
#imp_global <- imp_global[order(-Gain_prom)]

#head(imp_global, 20)

#top20 <- imp_global[1:20]

#barplot(top20$Gain_prom,
#        names.arg = top20$Feature,
#        las = 2,
#        cex.names = 0.7,
#        main = "Importancia promedio de variables (todos los modelitos)",
#        ylab = "Gain promedio")

In [ ]:
Sys.time()